# Variables

In [0]:
HOST = '127.0.0.1'        #    <== CHANGE THIS

# the host must be reachable from google VM


######################################################################################################

NMAP_DIR = '/content/NMAP'
DIRB_DIR = '/content/DIRB'
NIKTO_DIR = '/content/NIKTO'
WORDLISTS_DIR = '/content/WORDLISTS'

HTTP_WORDLIST = WORDLISTS_DIR + '/' + 'custom_url_wordlist.txt'
HTTP_EXTENSIONS_FILE = WORDLISTS_DIR + '/' + 'custom_extensions_common.txt'


# Folder hierarchy 

In [0]:
%cd /content/
!mkdir {NMAP_DIR}
!mkdir {DIRB_DIR}
!mkdir {NIKTO_DIR}
!mkdir {WORDLISTS_DIR}

# Setup

In [0]:
# nmap
!apt-get install nmap

# searchsploit
!git clone https://github.com/offensive-security/exploitdb.git /opt/exploitdb
!ln -sf /opt/exploitdb/searchsploit /usr/local/bin/searchsploit

# dirb
!apt-get install dirb

# nikto
!apt-get install nikto

# xmllint for searchsploit (and not only)
!apt-get install libxml2-utils

# wordlists
!curl -o "$HTTP_WORDLIST" https://raw.githubusercontent.com/Leviathan36/kaboom/master/kaboom/custom_url_wordlist.txt
!curl -o "$HTTP_EXTENSIONS_FILE" https://raw.githubusercontent.com/Leviathan36/kaboom/master/kaboom/custom_extensions_common.txt


# Functions

In [0]:
# This function print the number of port for the open services

import lxml.etree as xml

def service_ports (name, prot='TCP', state='open', ssl_on=False):
  root = xml.parse("/content/NMAP/syn_scan.xml")
  
  if prot == 'UDP':
    state = 'open|filtered'
   
  if ssl_on == False:
    path = ".//port[state[@state=" + "'" + state + "'" + "] and service[@name=" + "'" + name + "'" + "]]"
  elif ssl_on == True:
    path = ".//port[state[@state=" + "'" + state + "'" + "] and service[@name=" + "'" + name + "'" + " and @tunnel='ssl']]"
  else:
    raise TypeError('Invalid type for SSL parameter')
    
  ports = []
  for i in root.xpath(path):
    ports.append(i.attrib['portid'])
  
  return ports

# Info Gathering

## Port scan

### TCP-syn scan

In [0]:
!nmap -vvv -oA "{NMAP_DIR}/syn_scan" -PE -PS80,443,22,25,110,445 -PU -PP -PA80,443,22,25,110,445 -sS -p- -sV --allports -O --fuzzy --script "(default or auth or vuln or exploit) and not http-enum" {HOST}

### UDP scan

In [0]:
!nmap -vvv -oA "{NMAP_DIR}/udp_scan" -PE -PS80,443,22,25,110,445 -PU -PP -PA80,443,22,25,110,445 -sU --top-ports 200 -sV --allports {HOST}

# Web assessment

## HTTP

### Dirb

In [0]:
for PORT in service_ports('http'):
  !dirb "http://{HOST}:{PORT}/" {HTTP_WORDLIST} -r -l -o "{DIRB_DIR}/dirb_{PORT}.txt" -x {HTTP_EXTENSIONS_FILE} -z 200

### Nikto

In [0]:
for PORT in service_ports('http'):
  !nikto -Display PV -nolookup -ask no -Format htm -host "{HOST}:{PORT}" -output "{NIKTO_DIR}/nikto_{PORT}.html" -Plugins "ms10_070;report_html;embedded;cookies;put_del_test;outdated;drupal(0:0);clientaccesspolicy;msgs;httpoptions;negotiate;parked;favicon;apache_expect_xss;headers" -Tuning 4890bcde

## HTTPS

#### Pure HTTPS

### Dirb

In [0]:
for PORT in service_ports('https'):
  !dirb "https://{HOST}:{PORT}/" {HTTP_WORDLIST} -r -l -o "{DIRB_DIR}/dirb_https_{PORT}.txt" -x {HTTP_EXTENSIONS_FILE} -z 200

### Nikto

In [0]:
for PORT in service_ports('https'):
  !nikto -ssl -port {PORT} -Display PV -nolookup -ask no -Format htm -host {HOST} -output "{NIKTO_DIR}/nikto_https_{PORT}.html" -Plugins "ms10_070;report_html;embedded;cookies;put_del_test;outdated;drupal(0:0);clientaccesspolicy;msgs;httpoptions;negotiate;parked;favicon;apache_expect_xss;ssl;headers" -Tuning 4890bcde

#### SSL/HTTP

### Dirb

In [0]:
for PORT in service_ports('http', ssl_on=True):
  !dirb "https://{HOST}:{PORT}/" {HTTP_WORDLIST} -r -l -o "{DIRB_DIR}/dirb_https_{PORT}.txt" -x {HTTP_EXTENSIONS_FILE} -z 200

### Nikto

In [0]:
for PORT in service_ports('http', ssl_on=True):
  !nikto -ssl -port {PORT} -Display PV -nolookup -ask no -Format htm -host {HOST} -output "{NIKTO_DIR}/nikto_https_{PORT}.html" -Plugins "ms10_070;report_html;embedded;cookies;put_del_test;outdated;drupal(0:0);clientaccesspolicy;msgs;httpoptions;negotiate;parked;favicon;apache_expect_xss;ssl;headers" -Tuning 4890bcde

# Vulnerability search

## Exploit-DB -- Searchsploit

### TCP services

In [0]:
!searchsploit --www --nmap "{NMAP_DIR}/syn_scan.xml"

### UDP services

In [0]:
!searchsploit --www --nmap "{NMAP_DIR}/udp_scan.xml"

## Nmap result browsing

### TCP services

In [0]:
!xmllint --xpath "//table[elem[text()='VULNERABLE' and @key='state']]/@key" "{NMAP_DIR}/syn_scan.xml" 2> /dev/null | tr " " "\n" | cut -f2 -s -d'"' | awk -F "CVE-" '{printf "search cve:" ; print $2}'

### UDP services

In [0]:
!xmllint --xpath "//table[elem[text()='VULNERABLE' and @key='state']]/@key" "{NMAP_DIR}/udp_scan.xml" 2> /dev/null | tr " " "\n" | cut -f2 -s -d'"' | awk -F "CVE-" '{printf "search cve:" ; print $2}'

# Offline password cracking

## Variables

In [0]:
WORDLISTS_DIR = '/content/WORDLISTS'
HASHES_DIR = '/content/HASHES'

HASH_FILE = HASHES_DIR + 'hash_file.txt'    # THE FILE TO DECRYPT            #    <== CHANGE THIS
HASHCAT_WORDLIST = WORDLISTS_DIR + '/fasttrack.txt'    # THE WORDLIST TO USE #    <== CHANGE THIS

## Setup

### Folder hierarchy

In [0]:
# create WORDLISTS directory if it does not already exist

%cd '/content/'
!mkdir -p {WORDLISTS_DIR}
!mkdir {HASHES_DIR}

### Install Hashcat

In [0]:
!apt-get install hashcat

### Downloading wordlists

In [0]:
# fasttrack
!curl -o "{WORDLISTS_DIR}/fasttrack.txt" https://raw.githubusercontent.com/Leviathan36/kaboom/master/kaboom/fasttrack.txt

In [0]:
# rockyou
!curl -o "{WORDLISTS_DIR}/rockyou.txt.tar.gz" https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Leaked-Databases/rockyou.txt.tar.gz
!tar -xvzf "{WORDLISTS_DIR}/rockyou.txt.tar.gz" -C {WORDLISTS_DIR}

## Password Attacks

### sha512crypt (6), SHA512 (Unix)

In [0]:
!hashcat -m1800 -a0  {HASH_FILE} {HASHCAT_WORDLIST}

### NTLM

In [0]:
!hashcat -m1000 -a0  {HASH_FILE} {HASHCAT_WORDLIST}

### MySQL4.1/MySQL5

In [0]:
!hashcat -m300 -a0  {HASH_FILE} {HASHCAT_WORDLIST}